In [29]:
import requests
import pandas as pd
import numpy as np
import re


class SpaceXData:
    def getAllLaunches(self):
        url = "https://api.spacexdata.com/v3/launches"
        res = self.__getData(url)
        return self.__passResponseToDataFrame(res)

    def getAllLaunchPads(self):
        url = "https://api.spacexdata.com/v3/launchpads"
        res = self.__getData(url)
        return self.__passResponseToDataFrame(res)

    def __passResponseToDataFrame(self, data):
        return pd.read_json(data.text)

    def __getData(self, url):
        response = requests.request("GET", url, headers={}, data={})
        return response


class Methods:

    def getTheYearWithMoreLaunches(self, df):
        if not set(['launch_year', 'flight_number']).issubset(df.columns):
            raise ValueError("Dataframe hasn't the necessaries columns")
        result = df.groupby(['launch_year']).count().query(
            'flight_number==flight_number.max()')
        return result.loc[:, ['flight_number']]

    def getTheLaunchPadWithMoreLaunches(self, df):
        if not set(['name', 'attempted_launches']).issubset(df.columns):
            raise ValueError("Dataframe hasn't the necessaries columns")
        result = df.query('attempted_launches==attempted_launches.max()')
        return result.loc[:, ['name', 'attempted_launches']]

    def getTotalLaunches(self, df, initial_year, final_year):
        ini_year = int(initial_year)
        fin_year = int(final_year)
        array_of_years = sorted([ini_year, fin_year])
        array_of_str = list(map(lambda x: str(x), array_of_years))

        if not set(['launch_year', 'flight_number']).issubset(df.columns):
            raise ValueError("Dataframe hasn't the necessaries columns")

        for year in array_of_str:
            match = re.match(r'.*([0-2][0-9]{3})', year)
            if match is None:
                raise ValueError('Value is not  an acceptable argument')
        res = df.groupby(['launch_year']).count()
        return res.loc[array_of_str[0]:array_of_str[1]]['flight_number'].sum()

In [30]:
spaceXData = SpaceXData()
methods = Methods()


class Test(unittest.TestCase):

    def test_get_the_year_with_more_launches(self):
        df = spaceXData.getAllLaunches()
        result = methods.getTheYearWithMoreLaunches(df)
        self.assertEqual(result.index[0], 2020)
        self.assertEqual(result.values[0][0], 25)

    def test_get_the_launchpad_with_more_launches(self):
        df = spaceXData.getAllLaunchPads()
        result = methods.getTheLaunchPadWithMoreLaunches(df)
        self.assertEqual(result['name'][1], 'CCAFS SLC 40')

    def test_get_total_launches(self):
        df = spaceData.getAllLaunches()
        result = methods.getTotalLaunches(df, 2019, 2021)
        self.assertEqual(result, 38)

    def test_incorrect_dataframe_in_get_the_year_with_more_launches(self):
        df_mock = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                               columns=['a', 'b', 'c'])
        with self.assertRaises(Exception) as context:
            methods.getTheYearWithMoreLaunches(df_mock)
            self.assertTrue(
                "Dataframe hasn't the necessaries columns"
                in context.exception)

    def test_incorrect_dataframe_in_get_launchpad_function(self):
        df_mock = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                               columns=['a', 'b', 'c'])
        with self.assertRaises(Exception) as context:
            methods.getTheLaunchPadWithMoreLaunches(df_mock)
            self.assertTrue(
                "Dataframe hasn't the necessaries columns"
                in context.exception)

    def test_incorrect_dataframe_in_get_total_launches_function(self):
        df_mock = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                               columns=['a', 'b', 'c'])
        with self.assertRaises(Exception) as context:
            methods.getTotalLaunches(df_mock)
            self.assertTrue(
                "Dataframe hasn't the necessaries columns"
                in context.exception)

res = unittest.main(argv=[''], verbosity=4, exit=False)
# if we want our notebook to stop processing due to failures, we need a cell itself to fail
assert len(res.result.failures) == 0


test_get_the_launchpad_with_more_launches (__main__.Test) ... ok
test_get_the_year_with_more_launches (__main__.Test) ... ok
test_get_total_launches (__main__.Test) ... ok
test_incorrect_dataframe_in_get_launchpad_function (__main__.Test) ... ok
test_incorrect_dataframe_in_get_the_year_with_more_launches (__main__.Test) ... ok
test_incorrect_dataframe_in_get_total_launches_function (__main__.Test) ... ok

----------------------------------------------------------------------
Ran 6 tests in 43.056s

OK


In [31]:
methods = Methods()
spaceData = SpaceXData()
all_launches = spaceData.getAllLaunches()
all_launch_pads = spaceData.getAllLaunchPads()

first_question = methods.getTheYearWithMoreLaunches(all_launches)
second_question = methods.getTheLaunchPadWithMoreLaunches(all_launch_pads)
third_question = methods.getTotalLaunches(all_launches, 2019, 2021)

third_question_to_df = pd.DataFrame({'total_sum':[third_question]}, index=[1])

# third_question = methods.getTotalLaunches(2019,2021)

In [32]:
first_question

,flight_number
launch_year,
2020,25


In [33]:
second_question

,name,attempted_launches
1,CCAFS SLC 40,61


In [35]:
third_question_to_df

,total_sum
1,38


In [36]:
writer = pd.ExcelWriter('challenge_result.xlsx', engine='xlsxwriter')

first_question.to_excel(writer, sheet_name="Questão_1")
second_question.to_excel(writer, sheet_name="Questão_2")
third_question_to_df.to_excel(writer, sheet_name="Questão_3")

writer.save()